# 5. ラベルなしでの事前学習

## 5.1 生成テキストモデルを評価する

### GPTを使ってテキストを生成する

In [ ]:
from GPTModel import GPTModel


In [ ]:
import torch
from GPTModel import GPTModel

GPT_CONFIG_124M = {
  "vocab_size": 50257,
  "context_length": 256,
  "emb_dim": 768,
  "n_heads": 12,
  "n_layers": 12,
  "drop_rate": 0.1,
  "qkv_bias": False,  
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

In [ ]:
# List 5-1

import tiktoken
from util import generate_text_simple

def text_to_token_ids(text, tokenizer):
  encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
  encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dim
  return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
  flat = token_ids.squeeze(0) # remove batch dim.
  return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
  model=model,
  idx=text_to_token_ids(start_context, tokenizer),
  max_new_tokens=10,
  context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


### テキスト生成の損失を計算する

In [ ]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,     1107, 588]])  #  "I really like"]
targets = torch.tensor ([[3626, 6100, 345],   # [" effort moves you",
                         [1107, 588, 11311]]) #  " really like chocolate"]

In [ ]:
with torch.no_grad():
  logits = model(inputs)

probas = torch.softmax(logits, dim= -1)
print(probas.shape)

In [ ]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

In [ ]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

In [ ]:
text_idx = 0
target_brabs_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_brabs_1)

text_idx = 1
target_brabs_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_brabs_2)



In [ ]:
log_probas = torch.log(torch.cat((target_brabs_1, target_brabs_2)))
print(log_probas)

In [ ]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

In [ ]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

In [ ]:
print("Logits shape:", logits.shape)
print("Targets shape:", targets.shape)

In [ ]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

In [ ]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

### 訓練データセットで損失を計算する